# Distance learning with higher order constraints

We want to consider the higher order case when we do metric learning algorithm.

We do it as adaptation of the main notebook from https://github.com/kedemdor/metric-learning-talk

## Distance learnign with classical binary constraints

#### Loss functions

In classification and regression problem loss functions are usually composed of a (often differential) transformation

A metric learning loss function combines three elements:
1. **The transformation function**: A differential transformation function $\phi(x): \mathbb{R}^n \rightarrow \mathbb{R}^d$ could either be a linear transformation, as we've seen in the Mahalanobis distance (i.e., $\phi(x) = Lx$ where $L \in \mathbb{R}^{d\times n}$), but it could also be a non-linear transformer (i.e. DNN or other feature ensembles) from an $n$-dimensional space to a $d$-dimensional space. The parameters of these functions w

2. **The selected distance function to use in the latent space**: Most often, it would be either the Euclidean distance or the Cosine distance (which may be more meaningful in higher dimensional data).

3. **The objective**: Using the provided supervision, the transformation function and the desired metric, we formalize an optimization problem. The specified supervision incorporated to a differentiable function to minimze or maximize.

### Example loss function: linear LMNN for binary relations

Let's take for example one of the more common metric learning loss function: Large Margin Nearest Neighbors (LMNN).
loss function of LMNN is described as:

$L
=
\large\min_\limits{L}~~~
\lambda\underbrace{\sum_{i,~j\in N_i} ||Lx_i-Lx_j||_2^2}_\text{pull target neighbors} ~+(1-\lambda)~
\underbrace{\sum_{i,~j\in N_i,~k\notin N_i} max(0, 1 + ||Lx_i-Lx_j||_2^2 - ||Lx_i-Lx_k||_2^2)}_\text{push away impostors from the neighborhood}$

In this case:
1. The transformation function is **linear** and defined by $L$: from $x\in \mathbb{R}^n$, to $Lx \in \mathbb{R}^d$.
2. The distance function in the latent space is **Euclidean**, optimizing the loss function on the squared L2 distance: $d(\vec{a},\vec{b})^2 = ||\vec{a}-\vec{b}||_2^2$.
3. The objective of this loss function is composed of two parts:
    * A penalty for the squared distance between data points and their target neighbors (i.e. similar / within the same class), causing the optimization to ***pull*** them closer together in the latent space.
    * A penalty for impostors (dissimilar / of different class) which are closer to the point than the target neighbors + a margin, causing the optimization to ***push*** them away from the neighborhood.
    * The tradeoff between the two parts of the optimization is managed by the $\lambda$ parameter, where $\lambda=0.5$ is equal weight to the terms (best $\lambda$ could be discovered by cross-validation).


### Example loss function: linear LMNN for ternary relations

Then we will just need to change the first constraint and introduce it into our method of Large Margin Nearest Neighbors (LMNN).
loss function of LMNN is described below:

$ L = \large\min_\limits{L}~~~
\lambda\underbrace{\sum_{i,~j, k\in N_i} ||L(x_i,x_j, x_k)||_2^2}_\text{pull target neighbors} ~+(1-\lambda)~
\underbrace{\sum_{i,~j\in N_i,~k\notin N_i} max(0, 1 + ||Lx_i-Lx_j||_2^2 - ||Lx_i-Lx_k||_2^2)}_\text{push away impostors from the neighborhood}$


In [1]:
# For this demo, we would use these packages
!pip install -q huggingface-hub==0.0.19
!pip install -q scikit-learn==1.0 delayed
!pip install -q sentence-transformers
!pip install -q pytorch-metric-learning[with-hooks]
!pip install -q umap-learn
!pip install -q umap-learn[plot]
!pip install -q metric-learn
!pip install -q altair

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 1.1.1 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.0.19 which is incompatible.
diffusers 0.31.0 requires huggingface-hub>=0.23.2, but you have huggingface-hub 0.0.19 which is incompatible.
peft 0.13.2 requires huggingface-hub>=0.17.0, but you have huggingface-hub 0.0.19 which is incompatible.
sentence-transformers 3.2.1 requires huggingface-hub>=0.20.0, but you have huggingface-hub 0.0.19 which is incompatible.
tokenizers 0.20.3 requires huggingface-hub<1.0,>=0.16.4, but you have huggingface-hub 0.0.19 which is incompatible.
transformers 4.46.2 requires huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 0.0.19 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.5 MB/s 

# Sample data preparation
Importing stanadrd data of texts.
1. Step 1: Sentence embedding. To get from the textual representation of the customer query to a vectorized representation, we will use some state-of-the-art BERT setentence embeddings. These general-domain transformers are used to project a textual sentence (usually up to 128 words) into a 300-768 dimension embedding which should preserve their semantic meaning. This would be used as our initial input for metric learning, where we would learn a bank-specific representation.We will use

* The Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks package.
* For this we will need
NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [5]:
import pandas as pd
import numpy as np
import random

def read_dataset(set_name: str, show_samples:int=0 , random_state:int=42):
    # Retrieve the banking data from the github repository.
    url = f"https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/{set_name}.csv"
    dataset = pd.read_csv(url)
    # Describe the dataset.
    print(f"There's a total of [{len(dataset)}] texts " +
          f"across [{len(set(dataset['category']))}] categories in the [{set_name}] set.\n")
    # Show an example of the dataset.
    if show_samples:
        print("Here are some examples:\n------------------------")
        for _, row in dataset.sample(show_samples, random_state=random_state).iterrows():
            print(f"category: {row.category}\ntext: {row.text}\n")
    # Return the read dataset.
    return dataset

train_df = read_dataset("train", show_samples=4)
test_df = read_dataset("test", show_samples=4)


import pandas as pd

# Sample a number of sentences per category.
def sample_and_aggregate_dataset(df:pd.DataFrame, samples_per_category:int=20,
                                 random_state:int=42, verbose:bool=False):
  """ Aggregating on category, sampling a number of queries per category. """
  random.seed(random_state)
  agg_df = pd.DataFrame(
    df.groupby("category").agg(
        lambda corpus: random.sample(list(corpus), min([samples_per_category, len(corpus)])))
      .to_records())
  return agg_df

train_agg_df = sample_and_aggregate_dataset(train_df, samples_per_category=10, verbose=True)
test_agg_df = sample_and_aggregate_dataset(test_df, samples_per_category=2000, verbose=True)

print(f"Total number of points in train set reduced to [{sum(train_agg_df.text.map(len))}].")
print(f"Total number of points in test set reduced to [{sum(test_agg_df.text.map(len))}].")


There's a total of [10003] texts across [77] categories in the [train] set.

Here are some examples:
------------------------
category: change_pin
text: Is it possible for me to change my PIN number?

category: declined_card_payment
text: I'm not sure why my card didn't work

category: top_up_failed
text: I don't think my top up worked

category: card_payment_fee_charged
text: Can you explain why my payment was charged a fee?

There's a total of [3080] texts across [77] categories in the [test] set.

Here are some examples:
------------------------
category: card_linking
text: How do I link this new card?

category: card_swallowed
text: How do I retrieve my card from the machine?

category: verify_source_of_funds
text: I want to know where the funds come from.

category: automatic_top_up
text: I just activated auto top-up, but it is not letting me enable it. Why not?

Total number of points in train set reduced to [770].
Total number of points in test set reduced to [3080].


In [ ]:


# 1. Step 1: Sentence embedding.


device = "cuda"  # Change to "cpu" if running on CPU or "cuda" if you run with GPU.
# Loading the pretrained sentence BERT model.
from sentence_transformers import SentenceTransformer
# STSB = Sentence Transformer Siamese BERT.
# Note: Recently I've received error that the download sometimes fails.
# If that happens, just make sure you're outside of the VPN & retry.
sbert_model = SentenceTransformer('stsb-mpnet-base-v2', device=device)

In [ ]:
# Embed the samples in with the sentence2vec transformer. May take a 4-8 minutes if on CPU...
train_agg_df["sentence2vec"] = train_agg_df["text"].map(sbert_model.encode)
test_agg_df["sentence2vec"] = test_agg_df["text"].map(sbert_model.encode)

### Step 2.  Evaluating the embedding

Before we begin with the metric learning, let's first evaluate our initial general-purpose embeddings. There are different ways of measuring embeddings, but a common practice is to look into the neighborhoods and see if the nearby neighbors are semantically similar. Throughout the notebook, we will use three ways to evaluate the embedding.
For now we can skip it...


In [ ]:
from sklearn.manifold import TSNE

embeddings_tsne = TSNE(random_state=42, init='pca', learning_rate='auto').fit_transform(X=np.concatenate([X_train,X_test]))
vis_df["embeddings_tsne_0"] = embeddings_tsne[:,0]
vis_df["embeddings_tsne_1"] = embeddings_tsne[:,1]

render_embedding(vis_df, title="query =[STSB]=> 768 =[TSNE]=> 2 (Better!)",
                 coord_0_col='embeddings_tsne_0',
                 coord_1_col='embeddings_tsne_1')

## Step 3.1: Linear metric learning
We will use LMNN, but feel free to [explore the package](http://contrib.scikit-learn.org/metric-learn/metric_learn.html#supervised-learning-algorithms) and try other types of linear transformations, such as NCA or ITML.

Also, we're reducing the number of components in the new representation from 768 to 128, for two main reasons. First, it's faster to learn a transformation matrix of size (768 x 128) than it is of size (768 x 768). But more importantly, the core reasoning behind using a lower dimension is that if we used 768 features to represent the variance of a general purpose sentence, you don't need that many feaures to differentiate between the 77 categories of banking queries. And third, because we a small number of data points, using a smaller dimension in the latent space helps us avoid overfitting.



We train the LMNN transformer with `verbose=True` so you could follow up on what happens. The will see two interesting details:
* The **objective**, which is the same objective function described above, which with gradient descend we will attempt to lower every iteration.
* The **active constraints**, which are how many "impostors" are there inside neighborhoods they shouldn't be a part of. The lower this number is, the more "pure" and homogeneous the neighbors after the transformation.

When you attempt to beat this baseline, try different values for `n_components` (number of dimensions in the output representation), `k` (how many neighbors should we take into account when searching for impostors) and `regularization` which determines the ratio between pushing and pulling.

If your attempt doesn't converge, try increasing the `learn_rate` or increase the `max_iter`.


In [6]:
from metric_learn import LMNN
lmnn = LMNN(n_components=192, k=5,
            learn_rate=1e-2, max_iter=2000, random_state=42,
            verbose=True).fit(X_train, y_train)


#### Changing active constraints in the function

Now we want to change this and write another LMNN function in this library
K. Q. Weinberger, J. Blitzer, L. K. Saul. Distance Metric Learning for Large Margin Nearest Neighbor Classification. NIPS 2005.
http://contrib.scikit-learn.org/metric-learn/generated/metric_learn.LMNN.html


So now we can use the constraint of higher order in the data and use the specific function on optimisation:

See more details in the simulation above
**iter | objective | objective difference | active constraints | learning rate**


We will look into the source code of the standard LMNN_classic
and will add LMNN_higher_orer


In [ ]:
class LMNN_classic(MahalanobisMixin, TransformerMixin):
  """Large Margin Nearest Neighbor (LMNN)

  LMNN learns a Mahalanobis distance metric in the kNN classification
  setting. The learned metric attempts to keep close k-nearest neighbors
  from the same class, while keeping examples from different classes
  separated by a large margin. This algorithm makes no assumptions about
  the distribution of the data.

  Read more in the :ref:`User Guide <lmnn>`.

  Parameters
  ----------
  init : string or numpy array, optional (default='auto')
    Initialization of the linear transformation. Possible options are
    'auto', 'pca', 'identity', 'random', and a numpy array of shape
    (n_features_a, n_features_b).

    'auto'
      Depending on ``n_components``, the most reasonable initialization
      will be chosen. If ``n_components <= n_classes`` we use 'lda', as
      it uses labels information. If not, but
      ``n_components < min(n_features, n_samples)``, we use 'pca', as
      it projects data in meaningful directions (those of higher
      variance). Otherwise, we just use 'identity'.

    'pca'
      ``n_components`` principal components of the inputs passed
      to :meth:`fit` will be used to initialize the transformation.
      (See `sklearn.decomposition.PCA`)

    'lda'
      ``min(n_components, n_classes)`` most discriminative
      components of the inputs passed to :meth:`fit` will be used to
      initialize the transformation. (If ``n_components > n_classes``,
      the rest of the components will be zero.) (See
      `sklearn.discriminant_analysis.LinearDiscriminantAnalysis`)

    'identity'
      If ``n_components`` is strictly smaller than the
      dimensionality of the inputs passed to :meth:`fit`, the identity
      matrix will be truncated to the first ``n_components`` rows.

    'random'
      The initial transformation will be a random array of shape
      `(n_components, n_features)`. Each value is sampled from the
      standard normal distribution.

    numpy array
      n_features_b must match the dimensionality of the inputs passed to
      :meth:`fit` and n_features_a must be less than or equal to that.
      If ``n_components`` is not None, n_features_a must match it.

  n_neighbors : int, optional (default=3)
    Number of neighbors to consider, not including self-edges.

  min_iter : int, optional (default=50)
    Minimum number of iterations of the optimization procedure.

  max_iter : int, optional (default=1000)
    Maximum number of iterations of the optimization procedure.

  learn_rate : float, optional (default=1e-7)
    Learning rate of the optimization procedure

  tol : float, optional (default=0.001)
    Tolerance of the optimization procedure. If the objective value varies
    less than `tol`, we consider the algorithm has converged and stop it.

  verbose : bool, optional (default=False)
    Whether to print the progress of the optimization procedure.

  regularization: float, optional (default=0.5)
    Relative weight between pull and push terms, with 0.5 meaning equal
    weight.

  preprocessor : array-like, shape=(n_samples, n_features) or callable
    The preprocessor to call to get tuples from indices. If array-like,
    tuples will be formed like this: X[indices].

  n_components : int or None, optional (default=None)
    Dimensionality of reduced space (if None, defaults to dimension of X).

  random_state : int or numpy.RandomState or None, optional (default=None)
    A pseudo random number generator object or a seed for it if int. If
    ``init='random'``, ``random_state`` is used to initialize the random
    transformation. If ``init='pca'``, ``random_state`` is passed as an
    argument to PCA when initializing the transformation.

  k : Renamed to n_neighbors. Will be deprecated in 0.7.0

  Attributes
  ----------
  n_iter_ : `int`
    The number of iterations the solver has run.

  components_ : `numpy.ndarray`, shape=(n_components, n_features)
    The learned linear transformation ``L``.

  Examples
  --------

  >>> import numpy as np
  >>> from metric_learn import LMNN
  >>> from sklearn.datasets import load_iris
  >>> iris_data = load_iris()
  >>> X = iris_data['data']
  >>> Y = iris_data['target']
  >>> lmnn = LMNN(n_neighbors=5, learn_rate=1e-6)
  >>> lmnn.fit(X, Y, verbose=False)

  References
  ----------
  .. [1] K. Q. Weinberger, J. Blitzer, L. K. Saul. `Distance Metric
         Learning for Large Margin Nearest Neighbor Classification
         <http://papers.nips.cc/paper/2795-distance-metric\
         -learning-for-large-margin-nearest-neighbor-classification>`_. NIPS
         2005.
  """


[docs]

  def __init__(self, init='auto', n_neighbors=3, min_iter=50, max_iter=1000,
               learn_rate=1e-7, regularization=0.5, convergence_tol=0.001,
               verbose=False, preprocessor=None,
               n_components=None, random_state=None, k='deprecated'):
    self.init = init
    if k != 'deprecated':
      warnings.warn('"num_chunks" parameter has been renamed to'
                    ' "n_chunks". It has been deprecated in'
                    ' version 0.6.3 and will be removed in 0.7.0'
                    '', FutureWarning)
      n_neighbors = k
    self.k = 'deprecated'  # To avoid no_attribute error
    self.n_neighbors = n_neighbors
    self.min_iter = min_iter
    self.max_iter = max_iter
    self.learn_rate = learn_rate
    self.regularization = regularization
    self.convergence_tol = convergence_tol
    self.verbose = verbose
    self.n_components = n_components
    self.random_state = random_state
    super(LMNN, self).__init__(preprocessor)




[docs]

  def fit(self, X, y):
    k = self.n_neighbors
    reg = self.regularization
    learn_rate = self.learn_rate

    X, y = self._prepare_inputs(X, y, dtype=float,
                                ensure_min_samples=2)
    num_pts, d = X.shape
    output_dim = _check_n_components(d, self.n_components)
    unique_labels, label_inds = np.unique(y, return_inverse=True)
    if len(label_inds) != num_pts:
      raise ValueError('Must have one label per point.')
    self.labels_ = np.arange(len(unique_labels))

    self.components_ = _initialize_components(output_dim, X, y, self.init,
                                              self.verbose,
                                              random_state=self.random_state)
    required_k = np.bincount(label_inds).min()
    if self.n_neighbors > required_k:
      raise ValueError('not enough class labels for specified k'
                       ' (smallest class has %d)' % required_k)

    target_neighbors = self._select_targets(X, label_inds)

    # sum outer products
    dfG = _sum_outer_products(X, target_neighbors.flatten(),
                              np.repeat(np.arange(X.shape[0]), k))

    # initialize L
    L = self.components_

    # first iteration: we compute variables (including objective and gradient)
    #  at initialization point
    G, objective, total_active = self._loss_grad(X, L, dfG, k,
                                                 reg, target_neighbors,
                                                 label_inds)

    it = 1  # we already made one iteration

    if self.verbose:
      print("iter | objective | objective difference | active constraints",
            "| learning rate")

    # main loop
    for it in range(2, self.max_iter):
      # then at each iteration, we try to find a value of L that has better
      # objective than the previous L, following the gradient:
      while True:
        # the next point next_L to try out is found by a gradient step
        L_next = L - learn_rate * G
        # we compute the objective at next point
        # we copy variables that can be modified by _loss_grad, because if we
        # retry we don t want to modify them several times
        (G_next, objective_next, total_active_next) = (
            self._loss_grad(X, L_next, dfG, k, reg, target_neighbors,
                            label_inds))
        assert not np.isnan(objective)
        delta_obj = objective_next - objective
        if delta_obj > 0:
          # if we did not find a better objective, we retry with an L closer to
          # the starting point, by decreasing the learning rate (making the
          # gradient step smaller)
          learn_rate /= 2
        else:
          # otherwise, if we indeed found a better obj, we get out of the loop
          break
      # when the better L is found (and the related variables), we set the
      # old variables to these new ones before next iteration and we
      # slightly increase the learning rate
      L = L_next
      G, objective, total_active = G_next, objective_next, total_active_next
      learn_rate *= 1.01

      if self.verbose:
        print(it, objective, delta_obj, total_active, learn_rate)

      # check for convergence
      if it > self.min_iter and abs(delta_obj) < self.convergence_tol:
        if self.verbose:
          print("LMNN converged with objective", objective)
        break
    else:
      if self.verbose:
        print("LMNN didn't converge in %d steps." % self.max_iter)

    # store the last L
    self.components_ = L
    self.n_iter_ = it
    return self



  def _loss_grad(self, X, L, dfG, k, reg, target_neighbors, label_inds):
    # Compute pairwise distances under current metric
    Lx = L.dot(X.T).T

    # we need to find the furthest neighbor:
    Ni = 1 + _inplace_paired_L2(Lx[target_neighbors], Lx[:, None, :])
    furthest_neighbors = np.take_along_axis(target_neighbors,
                                            Ni.argmax(axis=1)[:, None], 1)
    impostors = self._find_impostors(furthest_neighbors.ravel(), X,
                                     label_inds, L)

    g0 = _inplace_paired_L2(*Lx[impostors])

    # we reorder the target neighbors
    g1, g2 = Ni[impostors]
    # compute the gradient
    total_active = 0
    df = np.zeros((X.shape[1], X.shape[1]))
    for nn_idx in reversed(range(k)):  # note: reverse not useful here
      act1 = g0 < g1[:, nn_idx]
      act2 = g0 < g2[:, nn_idx]
      total_active += act1.sum() + act2.sum()

      targets = target_neighbors[:, nn_idx]
      PLUS, pweight = _count_edges(act1, act2, impostors, targets)
      df += _sum_outer_products(X, PLUS[:, 0], PLUS[:, 1], pweight)

      in_imp, out_imp = impostors
      df -= _sum_outer_products(X, in_imp[act1], out_imp[act1])
      df -= _sum_outer_products(X, in_imp[act2], out_imp[act2])

    # do the gradient update
    assert not np.isnan(df).any()
    G = dfG * reg + df * (1 - reg)
    G = L.dot(G)
    # compute the objective function
    objective = total_active * (1 - reg)
    objective += G.flatten().dot(L.flatten())
    return 2 * G, objective, total_active

  def _select_targets(self, X, label_inds):
    target_neighbors = np.empty((X.shape[0], self.n_neighbors), dtype=int)
    for label in self.labels_:
      inds, = np.nonzero(label_inds == label)
      dd = euclidean_distances(X[inds], squared=True)
      np.fill_diagonal(dd, np.inf)
      nn = np.argsort(dd)[..., :self.n_neighbors]
      target_neighbors[inds] = inds[nn]
    return target_neighbors

  def _find_impostors(self, furthest_neighbors, X, label_inds, L):
    Lx = X.dot(L.T)
    margin_radii = 1 + _inplace_paired_L2(Lx[furthest_neighbors], Lx)
    impostors = []
    for label in self.labels_[:-1]:
      in_inds, = np.nonzero(label_inds == label)
      out_inds, = np.nonzero(label_inds > label)
      dist = euclidean_distances(Lx[out_inds], Lx[in_inds], squared=True)
      i1, j1 = np.nonzero(dist < margin_radii[out_inds][:, None])
      i2, j2 = np.nonzero(dist < margin_radii[in_inds])
      i = np.hstack((i1, i2))
      j = np.hstack((j1, j2))
      if i.size > 0:
        # get unique (i,j) pairs using index trickery
        shape = (i.max() + 1, j.max() + 1)
        tmp = np.ravel_multi_index((i, j), shape)
        i, j = np.unravel_index(np.unique(tmp), shape)
      impostors.append(np.vstack((in_inds[j], out_inds[i])))
    if len(impostors) == 0:
        # No impostors detected
        return impostors
    return np.hstack(impostors)




def _inplace_paired_L2(A, B):
  '''Equivalent to ((A-B)**2).sum(axis=-1), but modifies A in place.'''
  A -= B
  return np.einsum('...ij,...ij->...i', A, A)


def _count_edges(act1, act2, impostors, targets):
  imp = impostors[0, act1]
  c = Counter(zip(imp, targets[imp]))
  imp = impostors[1, act2]
  c.update(zip(imp, targets[imp]))
  if c:
    active_pairs = np.array(list(c.keys()))
  else:
    active_pairs = np.empty((0, 2), dtype=int)
  return active_pairs, np.array(list(c.values()))


def _sum_outer_products(data, a_inds, b_inds, weights=None):
  Xab = data[a_inds] - data[b_inds]
  if weights is not None:
    return np.dot(Xab.T, Xab * weights[:, None])
  return np.dot(Xab.T, Xab)

# Load hypergraph data from full raw arxiv data


In this data there are some constraints, which are of higher order, which we need to keep track of while we are doing the classification task.

In [ ]:
df = pd.read_csv('fullArxivWithOrcidAndCitations.csv', index_col=0, header=0, sep=',')

df = df.sort_values(by='created')
df['year'] = pd.DatetimeIndex(df['created']).year
df = df[pd.DatetimeIndex(df['created']).year >= 1992]


print('printing full dataset')

df.head(30)

# Classification testing: with and without higher order constraints

Now we want to test whether in the cases:
1. with just second oder constraints, on similarities, e.g. set S, D and without higher order constraints

2. with also higher order constraints from data, e.g. in sets S3, D3.

we get any improvements in the classfication while using distance learning algorithms for the classification.

We keep track of those constraints (S, D), or (S3, D3) while we are doing the classification tasks.